Definimos la función para la simulación

In [17]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=242):

    np.random.seed(random_seed)  # fijamos la semilla
    
    dt = T/N                     # incremento temporal
    dW = np.random.normal(scale = np.sqrt(dt), size=(n_sims, N))
    W = np.cumsum(dW, axis=1)
    
    time_step = np.linspace(dt, T, N)
    time_steps = np.broadcast_to(time_step, (n_sims, N))
    
    S_t = s_0 * np.exp((mu - 0.5 * sigma**2) * time_steps + sigma * W)   # Matriz (n_sim, T+1)
    S_t = np.insert(S_t, 0, s_0, axis=1)                 #Los reglones son trayectorias , columnas son tiempo
    
    return S_t

# Opciones Europeas

Una opción europea es aquella opción financiera donde el comprador de la misma podrá ejercerla solo cuando llegue el vencimiento del contrato.

Para este caso existen fórmulas analiticas para el precio de las opciones. Nosotros usaremos estas fórmulas para la simulación (revisar):

Call:

$$C(S_t,t)=N(d_1)S_t-N(d_2)Ke^{-r(T-t)}$$

Put:

$$P(S_t,t)=-N(-d_2)Ke^{-r(T-t)} - N(-d_1)S_t$$

donde

$$d_1=\frac{1}{\sigma \sqrt{T-t}}[ln(\frac{S_t}{K})+(r+\frac{\sigma^2}{2})(T-t)]$$


$$d_2=d_1-\sigma \sqrt{T-t}$$

Notación:

* S_0: precio inicial de la acción
* K: precio Strike
* r: tasa libre de riesgo anual
* $\sigma$: volatilidad
* T: Tiempo
* N: número de incrementos de las simulaciones

El precio de la opción es

Call:
    
$$max(S_T-K, 0)$$

Put:

$$max(K-S_T, 0)$$

In [18]:
import numpy as np
from scipy.stats import norm

In [19]:
S_0 = 100
K = 100
r = 0.05
sigma = 0.50
T = 1 # 1 año
N = 252 # 252 días en un año
dt = T / N # paso de tiempo
N_SIMS = 1000000 # numero de simulaciones 
discount_factor = np.exp(-r * T)

In [20]:
def black_scholes_analytical(S_0, K, T, r, sigma, type='call'):
    
    d1 = (np.log(S_0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S_0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if type == 'call':
        val = (S_0 * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1))
    elif type == 'put':
        val = (K * np.exp(-r * T) * norm.cdf(-d2, 0, 1) - S_0 * norm.cdf(-d1, 0, 1))
    else:
        raise ValueError('Wrong input for type!')
        
    return val

In [21]:
black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma, type='call')

21.79260421286685

Simulamos por MC el precio:

In [22]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, n_sims=N_SIMS, T=T, N=N)

Se usa el factor de descuento para calcular el valor presente del payoff

In [23]:
premium = discount_factor * np.mean(np.maximum(0, gbm_sims[:, -1] - K))

In [24]:
premium

21.749587324505637

Comentar: revisar librería Quantlib

## Opciones americanas

Las opciones americanas se caracterizan porque se pueden ejercer en cualquier momento del periodo $[0, T]$.

Al utilizar el método de Monte Carlo para opciones europeas, el periodo de tiempo se discretizó y se calculó el valor de los precios para el final de cada subperiodo. 

Se puede valuar una opción americana calculando el valor para subperiodos y eligiendo entre ellos el que da el valor óptimo. 

Al proceder así, se dice que la opción americana (que puede ejercerse en cualquier momento) se ha aproximado mediante una Opción Bermuda (que puede ejercerse solo en ciertos valores del tiempo).

In [25]:
import numpy as np
from scipy.stats import norm

Usamos la simulación MC que ya hemos utilizado.

In [26]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=242):

    np.random.seed(random_seed)  # fijamos la semilla
    
    dt = T/N                     # incremento temporal
    dW = np.random.normal(scale = np.sqrt(dt), size=(n_sims, N))
    W = np.cumsum(dW, axis=1)
    
    time_step = np.linspace(dt, T, N)
    time_steps = np.broadcast_to(time_step, (n_sims, N))
    
    S_t = s_0 * np.exp((mu - 0.5 * sigma**2) * time_steps + sigma * W)   # Matriz (n_sim, T+1)
    S_t = np.insert(S_t, 0, s_0, axis=1)                 #Los reglones son trayectorias , columnas son tiempo
    
    return S_t

Y podemos comparar con el cáluclo analítico

In [27]:
def black_scholes_analytical(S_0, K, T, r, sigma, type='call'):
    
    d1 = (np.log(S_0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S_0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if type == 'call':
        val = (S_0 * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1))
    elif type == 'put':
        val = (K * np.exp(-r * T) * norm.cdf(-d2, 0, 1) - S_0 * norm.cdf(-d1, 0, 1))
    else:
        raise ValueError('Wrong input for type!')
        
    return val

[Longstaff y Schwartz (2001)](https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf) presentan una manera de aproximar la esperanza condicionada en fragmentos del tiempo total, realizando en cada período una regresión por mínimos cuadrados sobre un conjunto finito de funciones de las variables de estado relevantes. La regresión es posible ya que se cuenta con la información de varios caminos al mismo tiempo (información transversal), por lo que la esperanza se estima realizando una regresión del valor del flujo de fondos que se obtiene por continuar descontado sobre los valores de las variables relevantes para los diferentes caminos de simulación. El valor estimado de la regresión es eficiente e insesgado de la esperanza condicionada y permite estimar con precisión la regla de parada óptima para la opción.

In [42]:
def lsmc_american_option(S_0, K, T, N, r, sigma, n_sims, option_type, poly_degree, random_seed=42):
    dt = T / N
    discount_factor = np.exp(-r * dt)

    gbm_simulations = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, 
                                   n_sims=n_sims, T=T, N=N,
                                   random_seed=random_seed)

    if option_type == 'call':
        payoff_matrix = np.maximum(
            gbm_simulations - K, np.zeros_like(gbm_simulations))
    elif option_type == 'put':
        payoff_matrix = np.maximum(
            K - gbm_simulations, np.zeros_like(gbm_simulations))

    value_matrix = np.zeros_like(payoff_matrix)
    value_matrix[:, -1] = payoff_matrix[:, -1]

    for t in range(N - 1, 0, -1):
        regression = np.polyfit(
            gbm_simulations[:, t], value_matrix[:, t + 1] * discount_factor, poly_degree)
        continuation_value = np.polyval(regression, gbm_simulations[:, t])
        value_matrix[:, t] = np.where(payoff_matrix[:, t] > continuation_value,
                                      payoff_matrix[:, t],
                                      value_matrix[:, t + 1] * discount_factor)

    option_premium = np.mean(value_matrix[:, 1] * discount_factor)
    return option_premium

In [43]:
S_0 = 36
K = 40
r = 0.06
sigma = 0.2
T = 1 # 1 year
N = 50 
dt = T / N 
n_sims = 10 ** 5 
discount_factor = np.exp(-r * dt)
option_type = 'put'
poly_degree = 5 

In [44]:
prima_opcion=lsmc_american_option(S_0, K, T, N, r, sigma, n_sims, option_type, poly_degree)

In [45]:
print(f'La prima de la opcion americana {option_type} es {option_premium:.3f}')

NameError: ignored

In [46]:
black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma, 
                         type='put')

3.84430779159684

In [48]:
european_call_price = black_scholes_analytical(S_0=S_0, K=K, T=T, 
                                               r=r, sigma=sigma)
american_call_price = lsmc_american_option(S_0=S_0, K=K, T=T, N=N, r=r, 
                                           sigma=sigma, n_sims=N_SIMS, 
                                           option_type='call', 
                                           poly_degree= POLY_DEGREE)
print(f"La prima de la call europea es {european_call_price:.3f}, y la de la call americana (usando {N_SIMS} simulaciones) es {american_call_price:.3f}")

NameError: ignored